# IR Lab SoSe 2024: Baseline Retrieval System

This jupyter notebook serves as baseline retrieval system that you can try to improve upon.
We will use the a corpus of scientific papers (title + abstracts) from the fields of information retrieval and natural language processing (the [IR Anthology](https://ir.webis.de/anthology/) and the [ACL Anthology](https://aclanthology.org/)). This serves Jupyter notebook only serves as retrieval system, i.e., it gets a set of information needs (topics) and a corpus as input and produces a run file as output. Please do evaluations in a new dedicated notebook.

### Step 1: Import Libraries

We will use [tira](https://www.tira.io/), an information retrieval shared task platform, for loading the (pre-built) retrieval index and [ir_dataset](https://ir-datasets.com/) to subsequently build a retrieval system with [PyTerrier](https://github.com/terrier-org/pyterrier), an open-source search engine.

Building your own index can be already one way that you can try to improve upon this baseline (if you want to focus on creating good document representations). Other ways could include reformulating queries or tuning parameters or building better retrieval pipelines.

In [33]:
# You only need to execute this cell if you are using Google Golab.
# If you use GitHub Codespaces, everything is already installed.
!pip3 install tira ir-datasets python-terrier

In [34]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
import pandas as pd

In [35]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

### Step 2: Load the Dataset and the Index

The type of the index object that we load is `<class 'jnius.reflect.org.terrier.structures.Index'>`, in fact a [Java class](http://terrier.org/docs/v3.6/javadoc/org/terrier/structures/Index.html) wrapped into Python. However, you do not need to worry about this: at this point, we will simply use the provided Index object to run procedures defined in Python.

In [36]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

### Step 3: Define the Retrieval Pipeline

We will define a BM25 retrieval pipeline as baseline. For details, see:

- [https://pyterrier.readthedocs.io](https://pyterrier.readthedocs.io)
- [https://github.com/terrier-org/ecir2021tutorial](https://github.com/terrier-org/ecir2021tutorial)

In [37]:
#base retrieval model for candidate set with bm25
#additional models tf and pl2 serve as re-rankers in pipeline
pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=["WMODEL:TF_IDF", "WMODEL:DirichletLM"])
#bm25 = pt.BatchRetrieve(index, wmodel="BM25")
#tf = pt.BatchRetrieve(index, wmodel="Tf")
#pl2 = pt.BatchRetrieve(index, wmodel="PL2")
#pipeline = bm25 >> (tf ** pl2)
#pipeline = (2* bm25 + tf + pl2).transform(topics)

### Step 4: Create the Run


In [38]:
print('First, we have a short look at the first three topics:')

pt_dataset.get_topics('text').head(3)

First, we have a short look at the first three topics:


,qid,query
0,1,retrieval system improving effectiveness
1,2,machine learning language identification
2,3,social media detect self harm


In [39]:
print('Now we do the retrieval...')
#run = (2* bm25 + tf + pl2).transform(pt_dataset.get_topics('text'))

#define empty feature dictionary
feature_dict = {}

#initial retrieve through pipeline
run = pipeline.transform(pt_dataset.get_topics('text'))
run = pipeline(pt_dataset.get_topics('text'))

# Convert the numpy arrays to lists
run['features'] = run['features'].apply(lambda x: x.tolist())

# Create new columns for each feature
run['Tf_Idf'] = run['features'].apply(lambda x: x[0])  # Assuming TF_IDF is the first feature
run['DirichletLM'] = run['features'].apply(lambda x: x[1])  # Assuming DirichletLM is the second feature

# compute reranking
reranked_run = run.sort_values(by=["Tf_Idf", "DirichletLM"], ascending=[False, True])
reranked_run.reset_index(drop=True, inplace=True)


print('Done. Here are the first 10 entries of the run')
#run.head(10)
reranked_run.head(10)

Now we do the retrieval...


Done. Here are the first 10 entries of the run


,qid,query,docid,rank,features,docno,score,Tf_Idf,DirichletLM
0,3,social media detect self harm,77305,785,"[24.56968647542429, 12.521474643088238]",2018.ecir_workshop-2018bir.10,14.300769,24.569686,12.521475
1,47,neural ranking for ecommerce product search,86669,866,"[23.6544536485414, 7.91952086784355]",2013.clef_conference-2013w.193,10.474266,23.654454,7.919521
2,45,analysis for android apps,58514,699,"[20.438095396442456, 8.396168968496946]",E85-1026,4.592333,20.438095,8.396169
3,3,social media detect self harm,103660,975,"[20.32673965376165, 7.538389474942977]",2015.wwwconf_conference-2015c.123,13.930027,20.326740,7.538389
4,3,social media detect self harm,90692,870,"[19.99468672858894, 8.715804585385392]",2011.ecir_conference-2011.20,14.113869,19.994687,8.715805
5,3,social media detect self harm,33018,922,"[19.462240628596923, 7.507640260681421]",W13-1609,14.010239,19.462241,7.507640
6,3,social media detect self harm,113194,757,"[19.096520995669373, 7.331128023613209]",2019.ir_journal-ir0volumeA22A12.0,14.400493,19.096521,7.331128
7,3,social media detect self harm,72806,904,"[18.873887669012444, 7.840583411951977]",2022.dravidianlangtech-1.5,14.083461,18.873888,7.840583
8,3,social media detect self harm,60921,932,"[18.029229229099517, 7.804481247024272]",2020.emnlp-main.107,13.975723,18.029229,7.804481
9,17,the ethics of artificial intelligence,55981,943,"[17.984992434316272, 10.480715772337518]",2021.conll-1.16,7.982643,17.984992,10.480716


### Step 5: Persist the run file for subsequent evaluations

The output of a prototypical retrieval system is a run file. This run file can later (optimally in a different notebook) be statistically evaluated.

In [40]:
persist_and_normalize_run(run, system_name='bm25-baseline', default_output='../runs')

The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".
